#  Análisis de Encuesta – Limpieza y Procesamiento Inicial

Este notebook realiza el tratamiento inicial de una base de datos de encuestas, donde se analizan variables tipo Likert y categóricas. El objetivo es preparar los datos para análisis exploratorio y visualizaciones.

---

##  1. Carga de datos y librerías


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



df_original = pd.read_excel("Prueba Tecnica Analista de Datos.xlsx", header=1)

df_original.head(10)

,# participante,Estado de la participación,Nivel general de satisfaccion,Eje 1- Organizacional,Orgullo por trabajar en la empresa,Recomendación,Beneficios,Bienestar,Balance,La empresa demuestra sensibilidad con las particularidades de la vida personal de sus empleados,...,Depresión Algún amigo cercano?,"27. Los trastornos de ANSIEDAD son estados en los que se experimentan sentimientos de miedo, temor o inquietud que no desaparecen y van empeorando con el tiempo, de tal forma que interfieren con la vida cotidiana, en las relaciones interpersonales, el trabajo o en estudio. Sus síntomas son sensación de peligro inminente, pánico o catástrofe, latidos cardíacos fuertes o rápidos, mareos, respiración acelerada, sudoración, temblores, problemas para concentrarse o para pensar en algo diferente al tema de preocupación, problemas gastrointestinales y problemas para conciliar el sueño. Teniendo en cuenta la descripción anterior, ¿Qué tan probable cree que esté padeciendo ANSIEDAD",Ansiedad Usted?,Ansiedad Algún familiar cercano?,Ansiedad Algún amigo cercano?,28. Modalidad de trabajo,29. Antigüedad en la organización,30. Tipo de contrato,31. Ciudad / Región donde labora,32. Area en la que labora
0,Participante 1,Participación completa,9,NaN,4,3,3,4,3,3,...,1.0,NaN,3.0,1.0,1.0,Fuera de la casa (oficina o calle),Entre 1 y 5 años,Término Indefinido,NaN,Administrativo
1,Participante 2,Participación completa,8,NaN,4,3,2,4,3,3,...,3.0,NaN,3.0,1.0,3.0,Fuera de la casa (oficina o calle),Entre 1 y 5 años,Término Indefinido,NaN,Sede externa
2,Participante 3,ha participado pero todavía no ha concluído,9,NaN,4,4,3,2,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Participante 4,Participación completa,10,NaN,4,4,4,4,4,4,...,2.0,NaN,1.0,1.0,1.0,Fuera de la casa (oficina o calle),Entre 6 y 10 años,Término Indefinido,NaN,Lideres
4,Participante 5,Participación completa,10,NaN,4,4,4,4,4,4,...,1.0,NaN,1.0,1.0,1.0,Fuera de la casa (oficina o calle),Entre 6 y 10 años,Término Indefinido,NaN,Cartera
5,Participante 6,Participación completa,8,NaN,4,3,2,2,3,4,...,3.0,NaN,4.0,2.0,2.0,Fuera de la casa (oficina o calle),Entre 1 y 5 años,Término Indefinido,NaN,Sede externa
6,Participante 7,ha participado pero todavía no ha concluído,10,NaN,4,4,4,4,3,4,...,3.0,NaN,3.0,3.0,3.0,En casa,Más de 20 años,Término Indefinido,NaN,Financiero
7,Participante 8,Participación completa,10,NaN,4,4,4,3,4,4,...,2.0,NaN,2.0,2.0,2.0,Fuera de la casa (oficina o calle),Entre 1 y 5 años,Término Indefinido,NaN,Proyectos
8,Participante 9,Participación completa,8,NaN,3,3,3,4,3,3,...,2.0,NaN,2.0,3.0,2.0,Mixto (en casa y fuera de la casa),Entre 1 y 5 años,Término Indefinido,NaN,Coordinación Administrativa
9,Participante 10,Participación completa,10,NaN,4,4,3,3,4,4,...,3.0,NaN,2.0,2.0,2.0,Mixto (en casa y fuera de la casa),Más de 20 años,Término Indefinido,NaN,Administrativo


---

##  2. Limpieza inicial: columnas y filas

Se eliminan columnas vacías, se renombran variables a formato limpio  y se filtran respuestas incompletas.


In [9]:
df = df_original.copy()

df.dropna(axis=1, how='all', inplace=True)

df.columns = (
    df.columns.astype(str)
    .str.strip()
    .str.lower()
    .str.replace(r"^\d+\._*", "", regex=True)  
    .str.replace(" ", "_")
    .str.replace("¿", "")
    .str.replace("?", "")
    .str.replace("/", "_")
    .str.replace("__", "_")
    .str.replace(r"^_", "", regex=True)  
)

df.drop(columns=["ciudad__región_donde_labora"], inplace=True)

df.reset_index(drop=True, inplace=True)

df = df[df["estado_de_la_participación"] != "ha participado pero todavía no ha concluído"]


print("Tamaño final del DataFrame:", df.shape)


Tamaño final del DataFrame: (215, 55)


In [10]:
print("Columnas del DataFrame:")

for i, col in enumerate(df.columns):
    print(f"{i+1:>2}. {col}")

Columnas del DataFrame:
 1. #_participante
 2. estado_de_la_participación
 3. nivel_general_de_satisfaccion
 4. orgullo_por_trabajar_en_la_empresa
 5. recomendación
 6. beneficios
 7. bienestar
 8. balance
 9. la_empresa_demuestra_sensibilidad_con_las_particularidades_de_la_vida_personal_de_sus_empleados
10. recursos
11. información
12. cargo
13. formacion
14. entrenamiento_cargo
15. evaluacion
16. comunicacion_con_otras_áreas
17. comunicación_con_el_jefe
18. trabajo_en_equipo_con_otras_áreas
19. trabajo_ene_quipo_con_mi_área
20. lideres_que_inspiran
21. lideres_que_forman
22. género
23. estado_civil
24. año_de_nacimiento
25. nivel_de_estudios
26. tipo_de_vivienda_donde_habita
27. cuántos_hijos__hijas_tiene
28. indique_el_rango_de_edad_de_su_primer_hijo_a
29. indique_el_rango_de_edad_de_su_segundo_hijo_a
30. indique_el_rango_de_edad_de_su_tercer_hijo_a
31. indique_el_rango_de_edad_de_su_cuarto_hijo_a
32. indique_el_rango_de_edad_de_su_quinto_hijo_a
33. formación_técnica_tecnológica
34.

---

##  3. Tratamiento de variables tipo Likert (1 a 10)

Estas variables son respuestas ordinales (pero tratadas como numéricas) utilizadas para medir percepciones, satisfacción, emociones, etc.


In [11]:
df_numeric = df.copy()  # se crea una copia para evitar modificar el original
df_numeric_test = df_numeric.apply(pd.to_numeric, errors='coerce')

likely_likert_cols = []
for col in df_numeric_test.columns:
    col_vals = df_numeric_test[col].dropna().unique()
    if len(col_vals) > 0 and np.all((col_vals >= 1) & (col_vals <= 10)):
        likely_likert_cols.append(col)

df[likely_likert_cols] = df[likely_likert_cols].apply(pd.to_numeric, errors='coerce').astype("Int64")

print("Resumen estadístico:")
df[likely_likert_cols].describe()


Resumen estadístico:


,nivel_general_de_satisfaccion,orgullo_por_trabajar_en_la_empresa,recomendación,beneficios,bienestar,balance,la_empresa_demuestra_sensibilidad_con_las_particularidades_de_la_vida_personal_de_sus_empleados,recursos,información,cargo,...,perro,pájaro,pez,otro,depresión_usted,depresión_algún_familiar_cercano,depresión_algún_amigo_cercano,ansiedad_usted,ansiedad_algún_familiar_cercano,ansiedad_algún_amigo_cercano
count,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,215.0,...,93.0,6.0,2.0,1.0,215.0,215.0,215.0,215.0,215.0,215.0
mean,9.213953,3.813953,3.725581,3.367442,3.32093,3.288372,3.646512,3.702326,3.525581,3.306977,...,1.0,1.0,1.0,1.0,1.906977,2.134884,2.186047,2.069767,2.055814,2.083721
std,1.164422,0.413319,0.467691,0.662663,0.738955,0.670196,0.534488,0.497418,0.586488,0.784605,...,0.0,0.0,0.0,<NA>,0.896878,0.929999,0.943641,0.94709,0.9103,0.958223
min,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,9.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,10.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,...,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
75%,10.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,1.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0
max,10.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,4.0,4.0


---

##  4. Limpieza de variables categóricas

Se estandariza texto (minúsculas, sin espacios extra) y se identifican valores nulos. Las categorías se dejan en texto para análisis posterior o visualización.


In [12]:
categorical_cols = df.select_dtypes(include="object").columns.tolist()

df[categorical_cols] = df[categorical_cols].apply(lambda col: col.str.strip().str.lower())

categorical_summary = {col: df[col].value_counts(dropna=False) for col in categorical_cols}

categorical_nulls = df[categorical_cols].isnull().sum().sort_values(ascending=False)

pd.DataFrame({
    "columna": categorical_nulls.index,
    "valores_nulos": categorical_nulls.values
})

,columna,valores_nulos
0,indique_el_rango_de_edad_de_su_quinto_hijo_a,214
1,indique_el_rango_de_edad_de_su_cuarto_hijo_a,206
2,indique_el_rango_de_edad_de_su_tercer_hijo_a,191
3,indique_el_rango_de_edad_de_su_segundo_hijo_a,130
4,indique_el_rango_de_edad_de_su_primer_hijo_a,73
5,#_participante,0
6,género,0
7,estado_de_la_participación,0
8,tipo_de_vivienda_donde_habita,0
9,nivel_de_estudios,0


---

##  5. Agrupación de bloques temáticos


Las variables fueron **agrupadas por bloques temáticos**, lo que facilita futuros análisis, tanto en Python como en Power BI:

- **Información personal**: edad, género, estado civil, número de hijos
- **Interés en formación**: formación técnica, idiomas, desarrollo personal, etc.
- **Mascotas**: tipo de mascota (o ninguna)
- **Salud mental**: niveles de ansiedad y depresión (autoevaluación, familia, amigos)
- **Datos laborales**: modalidad de trabajo, tipo de contrato, área, antigüedad
- **Satisfacción general**: nivel general de satisfacción
- **Ejes de percepción**:
  - Eje 1: Organizacional (beneficios, orgullo, balance vida-trabajo…)
  - Eje 2: Cargo (recursos, formación, evaluación…)
  - Eje 3: Liderazgo (comunicación, inspiración, trabajo en equipo…)

Esta estructura temática permitirá realizar análisis comparativos, construir visualizaciones agrupadas y aplicar modelos más fácilmente.

In [13]:
# 1. Información general del participante
cols_info_general = [
    "#_participante", "estado_de_la_participación", "género", "estado_civil",
    "año_de_nacimiento", "nivel_de_estudios", "tipo_de_vivienda_donde_habita",
    "cuántos_hijos__hijas_tiene",
    "indique_el_rango_de_edad_de_su_primer_hijo_a", "indique_el_rango_de_edad_de_su_segundo_hijo_a",
    "indique_el_rango_de_edad_de_su_tercer_hijo_a", "indique_el_rango_de_edad_de_su_cuarto_hijo_a",
    "indique_el_rango_de_edad_de_su_quinto_hijo_a"
]

# 2. Interés en formación
cols_intereses_formativos = [
    "formación_técnica_tecnológica", "formación_profesional_(pregrado_posgrado)", "idiomas",
    "habilidades_básicas_(excel,_power_point,_computación)", "desarrollo_personal",
    "formación_artística", "formación_para_emprendedores"
]

# 3. Mascotas
cols_mascotas = ["no_tengo_mascotas", "gato", "perro", "pájaro", "pez", "otro"]

# 4. Salud mental (Depresión y Ansiedad)
cols_salud_mental = [
    "depresión_usted", "depresión_algún_familiar_cercano", "depresión_algún_amigo_cercano",
    "ansiedad_usted", "ansiedad_algún_familiar_cercano", "ansiedad_algún_amigo_cercano"
]

# 5. Datos laborales
cols_laborales = [
    "modalidad_de_trabajo", "antigüedad_en_la_organización", "tipo_de_contrato", "area_en_la_que_labora"
]

# 6. Evaluación general
cols_satisfaccion = ["nivel_general_de_satisfaccion"]

# 7. Eje 1 - Organizacional
cols_eje_1 = [
    "orgullo_por_trabajar_en_la_empresa", "recomendación", "beneficios", "bienestar",
    "balance", "la_empresa_demuestra_sensibilidad_con_las_particularidades_de_la_vida_personal_de_sus_empleados"
]

# 8. Eje 2 - Cargo
cols_eje_2 = ["recursos", "información", "cargo", "formacion", "entrenamiento_cargo", "evaluacion"]

# 9. Eje 3 - Liderazgo
cols_eje_3 = [
    "comunicacion_con_otras_áreas", "comunicación_con_el_jefe", "trabajo_en_equipo_con_otras_áreas",
    "trabajo_ene_quipo_con_mi_área", "lideres_que_inspiran", "lideres_que_forman"
]

# 🧩 Reordenar DataFrame final
final_column_order = (
    cols_info_general +
    cols_intereses_formativos +
    cols_mascotas +
    cols_salud_mental +
    cols_laborales +
    cols_satisfaccion +
    cols_eje_1 +
    cols_eje_2 +
    cols_eje_3
)

df = df[final_column_order]


---

## 5. Estado final del DataFrame

Dataset **listo para análisis exploratorio** o para ser exportado a Power BI:

- ✅ Columnas limpias y con nombres uniformes (minúsculas, sin tildes ni símbolos)
- ✅ Se eliminaron columnas completamente vacías y filas con registros incompletos
- ✅ Las **variables tipo Likert** (1 a 10) fueron identificadas y convertidas a tipo entero
- ✅ Las **variables categóricas** fueron normalizadas (minusculizadas y sin espacios)
- ✅ Se eliminó la columna de ubicación (`ciudad__región_donde_labora`) por ser completamente nula



---

##  6. Exportación del DataFrame final

Una vez realizada toda la limpieza, transformación y agrupación de columnas temáticas, podemos exportar el DataFrame listo para análisis en Power BI u otras herramientas.

Usamos `to_excel()` para guardar el archivo en formato `.xlsx`:

In [14]:
# Exportar el DataFrame limpio a un archivo Excel 

df.to_excel("datos_limpios_para_powerbi.xlsx", index=False)
print("✅ Archivo exportado exitosamente: datos_limpios_para_powerbi.xlsx")

✅ Archivo exportado exitosamente: datos_limpios_para_powerbi.xlsx


---

##  7. Análisis exploratorio simple

En esta sección se realiza una primera exploración del dataset para comprender mejor la distribución de los datos y posibles relaciones entre variables.

###  Estadísticas descriptivas de variables tipo Likert

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 0 to 229
Data columns (total 55 columns):
 #   Column                                                                                           Non-Null Count  Dtype 
---  ------                                                                                           --------------  ----- 
 0   #_participante                                                                                   215 non-null    object
 1   estado_de_la_participación                                                                       215 non-null    object
 2   género                                                                                           215 non-null    object
 3   estado_civil                                                                                     215 non-null    object
 4   año_de_nacimiento                                                                                215 non-null    object
 5   nivel_de_estudios   

In [16]:
# Estadísticas descriptivas básicas
df[likely_likert_cols].describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
nivel_general_de_satisfaccion,215.0,9.21,1.16,2.0,9.0,10.0,10.0,10.0
orgullo_por_trabajar_en_la_empresa,215.0,3.81,0.41,2.0,4.0,4.0,4.0,4.0
recomendación,215.0,3.73,0.47,2.0,3.0,4.0,4.0,4.0
beneficios,215.0,3.37,0.66,1.0,3.0,3.0,4.0,4.0
bienestar,215.0,3.32,0.74,1.0,3.0,3.0,4.0,4.0
balance,215.0,3.29,0.67,2.0,3.0,3.0,4.0,4.0
la_empresa_demuestra_sensibilidad_con_las_particularidades_de_la_vida_personal_de_sus_empleados,215.0,3.65,0.53,1.0,3.0,4.0,4.0,4.0
recursos,215.0,3.7,0.5,2.0,3.0,4.0,4.0,4.0
información,215.0,3.53,0.59,2.0,3.0,4.0,4.0,4.0
cargo,215.0,3.31,0.78,1.0,3.0,3.0,4.0,4.0


In [17]:
for col in categorical_cols:
    print(f"🧩 {col}")
    print(df[col].value_counts(dropna=False), "\n")

🧩 #_participante
#_participante
participante 1      1
participante 2      1
participante 4      1
participante 5      1
participante 6      1
                   ..
participante 225    1
participante 226    1
participante 227    1
participante 228    1
participante 230    1
Name: count, Length: 215, dtype: int64 

🧩 estado_de_la_participación
estado_de_la_participación
participación completa    215
Name: count, dtype: int64 

🧩 género
género
femenino     142
masculino     73
Name: count, dtype: int64 

🧩 estado_civil
estado_civil
soltero (a)                      108
unión libre                       57
casado (a)                        37
separado (a) / divorciado (a)     11
viudo (a)                          2
Name: count, dtype: int64 

🧩 año_de_nacimiento
año_de_nacimiento
entre 1982 y 1994    81
entre 1965 y 1981    79
después de 1995      46
antes de 1964         9
Name: count, dtype: int64 

🧩 nivel_de_estudios
nivel_de_estudios
profesional                       68
técnico / tecnó

In [18]:
correlation_matrix = df[likely_likert_cols].corr()
correlation_matrix

,nivel_general_de_satisfaccion,orgullo_por_trabajar_en_la_empresa,recomendación,beneficios,bienestar,balance,la_empresa_demuestra_sensibilidad_con_las_particularidades_de_la_vida_personal_de_sus_empleados,recursos,información,cargo,...,perro,pájaro,pez,otro,depresión_usted,depresión_algún_familiar_cercano,depresión_algún_amigo_cercano,ansiedad_usted,ansiedad_algún_familiar_cercano,ansiedad_algún_amigo_cercano
nivel_general_de_satisfaccion,1.000000,0.325828,0.339990,0.400285,0.180502,0.219964,0.392384,0.296031,0.251964,0.214201,...,NaN,NaN,NaN,NaN,-0.132986,-0.005198,0.027396,-0.055971,0.019541,-0.078949
orgullo_por_trabajar_en_la_empresa,0.325828,1.000000,0.556556,0.421371,0.227005,0.295803,0.483558,0.433965,0.328159,0.277802,...,NaN,NaN,NaN,NaN,-0.147751,0.041276,0.053218,-0.038311,0.064988,0.015915
recomendación,0.339990,0.556556,1.000000,0.477647,0.215456,0.283464,0.582192,0.470773,0.357914,0.357983,...,NaN,NaN,NaN,NaN,-0.228245,0.074755,0.073871,-0.178116,0.069072,0.082786
beneficios,0.400285,0.421371,0.477647,1.000000,0.416510,0.360043,0.473980,0.375911,0.402539,0.312308,...,NaN,NaN,NaN,NaN,-0.240995,0.025357,0.024678,-0.167614,-0.010917,-0.048673
bienestar,0.180502,0.227005,0.215456,0.416510,1.000000,0.293467,0.229416,0.273831,0.245133,0.288686,...,NaN,NaN,NaN,NaN,0.024104,0.093107,0.074806,0.088042,0.084395,0.074066
balance,0.219964,0.295803,0.283464,0.360043,0.293467,1.000000,0.351125,0.328786,0.207025,0.399605,...,NaN,NaN,NaN,NaN,-0.102871,0.004777,-0.092618,-0.098102,-0.034165,-0.154192
la_empresa_demuestra_sensibilidad_con_las_particularidades_de_la_vida_personal_de_sus_empleados,0.392384,0.483558,0.582192,0.473980,0.229416,0.351125,1.000000,0.410876,0.371842,0.304537,...,NaN,NaN,NaN,NaN,-0.244380,0.030564,0.001293,-0.052596,0.079157,-0.051433
recursos,0.296031,0.433965,0.470773,0.375911,0.273831,0.328786,0.410876,1.000000,0.538797,0.414833,...,NaN,NaN,NaN,NaN,-0.146155,0.046795,0.018985,-0.104497,-0.004416,0.013315
información,0.251964,0.328159,0.357914,0.402539,0.245133,0.207025,0.371842,0.538797,1.000000,0.409360,...,NaN,NaN,NaN,NaN,-0.208663,-0.044909,-0.050857,-0.116800,-0.028945,-0.028774
cargo,0.214201,0.277802,0.357983,0.312308,0.288686,0.399605,0.304537,0.414833,0.409360,1.000000,...,NaN,NaN,NaN,NaN,-0.165086,-0.005779,-0.001761,-0.066687,0.021697,0.027810



## 8. Preguntas 

En esta sección se da respuesta a las preguntas planteadas en la prueba técnica 

####	¿Qué hallazgos relevantes encuentras en los datos?


El conjunto de datos entregado contiene información clave sobre la percepción de los empleados en múltiples dimensiones organizacionales, combinando variables numéricas y categóricas relacionadas con aspectos demográficos, laborales y emocionales. Este tipo de datos me permitió comprender en profundidad la experiencia del empleado en áreas como el liderazgo, el entorno de trabajo, las oportunidades de formación y la percepción del bienestar emocional.

Como parte del tratamiento inicial, identifique que existían aproximadamente 15 registros de empleados cuyo estado era "ha participado pero todavía no ha concluido", así como columnas irrelevantes o vacías que no aportaban valor analítico, de tal manera que fueron descartadas, garantizando así la integridad y claridad del conjunto de datos. Realicé además una verificación y clasificación de las variables según su tipo (categóricas o numéricas) para aplicar el tratamiento adecuado a cada una. Adicionalmente, organicé las columnas en ejes temáticos relevantes. Esta clasificación me permitió analizar el dataset de manera lógica y orientada a presentar resultados estructurados y que facilitaran la identificación de patrones significativos y su impacto sobre el nivel de satisfacción con la empresa. Como resultado del tratamiento inicial le di forma a una base de datos más limpia, óptima para su análisis visual en Power BI.

A partir del análisis realizado, identifique varios hallazgos relevantes. En primer lugar, una alta satisfacción general, con un promedio de 9,2, lo que muestra un ambiente laboral muy positivo. Respecto a la modalidad de trabajo, no hubo diferencias significativas.

En relación con el tipo de contrato, los empleados con contratos a término fijo o indefinido presentan mayores niveles de satisfacción, mientras que los temporales y aprendices muestran niveles inferiores. En cuanto a la salud mental, el 25,58 % de los empleados reportó niveles de depresión superiores o iguales a 3 (en una escala de 1 a 5), y un 32,56 % indicó niveles de ansiedad superiores o iguales a 3, lo que evidencia posibles riesgos emocionales dentro del entorno laboral.

El perfil general de los participantes muestra predominancia de personas nacidas entre 1982 y 1994, mayor proporción de mujeres, y niveles educativos principalmente profesionales o técnicos/tecnólogos. Además, una parte significativa tiene hijos y trabaja en el área administrativa o externa.

Finalmente, en cuanto a intereses de aprendizaje, se identificó como preferencia principal el desarrollo personal. La opción con menor interés fue la formación técnica/tecnológica, con diferencias notables según el nivel de escolaridad y la antigüedad del empleado.


####	¿Qué visualizaciones propones para presentar esta información?


Con el objetivo de presentar los resultados de manera clara, comprensible y orientada a la toma de decisiones, propongo el uso de visualizaciones simples e intuitivas que faciliten al cliente la interpretación de los hallazgos. Gráficas básicas pero efectivas, permiten resaltar de forma visual los aspectos más relevantes del análisis sin recurrir a explicaciones técnicas complejas.

El uso de elementos como indicadores, gráficas de barras, gráficos circulares, tarjetas informativas y filtros interactivos permite estructurar la información de forma accesible, favoreciendo una narrativa visual que guíe al cliente en la comprensión de los datos. 


#### 	¿Cuáles son tus principales conclusiones sobre los datos?


* Información general empleados

El perfil de los participantes se concentra en personas en edades entre 30 y 45 años en su mayoria mujeres, el nivel educativo predominante es profesional o técnico/tecnológico, y una gran cantidad de empleados tienen hijos. Esta diversidad permite analizar la experiencia laboral desde una perspectiva integral que incluye factores personales, familiares y socioeducativos.

* Intereses academicos

Hay un fuerte interés por el desarrollo personal y la adquisición de habilidades blandas, seguido por temas como idiomas y herramientas básicas. Esto sugiere una alta disposición al aprendizaje continuo y una oportunidad para diseñar planes de formación adaptados a los distintos perfiles y trayectorias dentro de la organización.

* Salud mental (Depresión y Ansiedad)

Una proporción significativa de empleados reporta síntomas de depresión (25,58 %) y ansiedad (32,56 %) en niveles iguales o superiores a 3 en una escala de 1 a 5. Poniendo en evidencia posibles riesgos psicosociales que deben ser atendidos mediante programas de apoyo emocional, prevención y acompañamiento en salud mental.

* Evaluación general

El nivel general de satisfacción es alto (promedio de 9,2), lo que indica un clima organizacional bueno. Existen diferencias sutiles entre grupos que podrían ser abordadas para fortalecer la experiencia laboral de manera equitativa.

* Ejes organizacionales (Eje 1, 2 y 3)

En conjunto, las dimensiones relacionadas con la organización, el cargo y el liderazgo presentan resultados positivos. Los empleados valoran el sentido de pertenencia, el acceso a recursos y la comunicación con sus líderes directos. Sin embargo, se ven oportunidades de mejora en aspectos como la inspiración de los líderes, la formación dentro del rol, y la colaboración entre áreas, las cuales se encontraban con un promedio inferior.